In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from timeit import default_timer
import time
import json
import numpy as np

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split = ['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

2022-11-27 23:46:14.059706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., tf.one_hot(label, depth = 10)

ds_train = ds_train.map(normalize_img, num_parallel_calls = tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [4]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [5]:
# Creating the Model...
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28)),
  tf.keras.layers.Dense(128, activation = 'relu'),
  tf.keras.layers.Dense(10)
])

In [6]:
# Compiling the Model
model.compile(
    optimizer = tf.keras.optimizers.Adam(1e-3),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
    metrics = [tf.keras.metrics.CategoricalAccuracy()],
)

In [7]:
class TimingCallback(tf.keras.callbacks.Callback):
    def __init__(self, logs = {}):
        self.logs = []
        
        self.start_list = []
        self.end_list = []
        self.difference_list = []
        
        self.start_train_time = 0.0
        self.end_train_time = 0.0
        
        self.start_batch_interference_list = []
        self.end_batch_interference_list = []
        self.difference_batch_interference_list = []
        
        self.final_training_loss = 0.0
        self.final_evaluation_accuracy = 0.0
        
    def on_train_begin(self, logs = {}):
        self.start_train_time = time.time()
    def on_train_end(self, logs = {}):
        self.end_train_time = time.time()
        
    def on_test_batch_begin(self, batch, logs = {}):
        self.starttime = time.time()
        self.start_batch_interference_list.append(self.starttime)
    def on_test_batch_end(self, batch, logs = {}):
        self.endtime = time.time()
        self.end_batch_interference_list.append(self.endtime)
        self.difference_batch_interference_list.append(self.endtime - self.starttime)
        
    def on_epoch_begin(self, epoch, logs = {}):
        self.starttime = time.time()
        self.start_list.append(self.starttime)
    def on_epoch_end(self, epoch, logs = {}):
        self.final_training_loss += logs["loss"]
        self.final_evaluation_accuracy += logs["categorical_accuracy"]
        self.endtime = time.time()
        self.end_list.append(self.endtime)
        self.difference_list.append(self.endtime - self.starttime)
            
cb = TimingCallback()

In [8]:
# Fitting the Model...

model.fit(
    ds_train,
    epochs = 10,
    validation_data = ds_test,
    callbacks = [cb]
)

Epoch 1/10
469/469 [==============================] - 3s 2ms/step - loss: 0.3554 - categorical_accuracy: 0.9033 - val_loss: 0.1960 - val_categorical_accuracy: 0.9438
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1629 - categorical_accuracy: 0.9536 - val_loss: 0.1431 - val_categorical_accuracy: 0.9564
Epoch 3/10
469/469 [==============================] - 1s 1ms/step - loss: 0.1168 - categorical_accuracy: 0.9668 - val_loss: 0.1076 - val_categorical_accuracy: 0.9671
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0909 - categorical_accuracy: 0.9736 - val_loss: 0.0966 - val_categorical_accuracy: 0.9716
Epoch 5/10
469/469 [==============================] - 1s 1ms/step - loss: 0.0727 - categorical_accuracy: 0.9793 - val_loss: 0.0890 - val_categorical_accuracy: 0.9715
Epoch 6/10
469/469 [==============================] - 1s 1ms/step - loss: 0.0606 - categorical_accuracy: 0.9826 - val_loss: 0.0794 - val_categorical_accuracy: 0.9758
Epoc

In [9]:
total_training_time = cb.end_train_time - cb.start_train_time
average_epoch_training_time = np.mean(cb.difference_list)
average_batch_interference_time = np.mean(cb.difference_batch_interference_list) * 1000
final_accuracy = np.mean(cb.final_evaluation_accuracy) * 10

metrics = {
    "model_name": "MLP",
    "framework_name": "TensorFlow",
    "dataset": "MNIST Digits",
    "task": "classification",
    "Total Training Time" : total_training_time, # in seconds
    "Final Training Loss" : cb.final_training_loss, 
    "Final Evaluation Accuracy" : final_accuracy,
    "Average Epoch Training Time" : average_epoch_training_time, # in seconds
    "Average Batch Inference Time" : average_batch_interference_time # in milliseconds
}

for key, value in metrics.items():
    print(f'{key} : {value}')

with open("output.json", "w") as outfile:
    json.dump(metrics, outfile)

model_name : MLP
framework_name : TensorFlow
dataset : MNIST Digits
task : classification
Total Training Time : 9.389873266220093
Final Training Loss : 1.0210097339004278
Final Evaluation Accuracy : 97.1313339471817
Average Epoch Training Time : 0.0013297557830810546
Average Batch Inference Time : 1.0042100013056887


In [10]:
model.evaluate(
    ds_train
)

469/469 [==============================] - 0s 825us/step - loss: 0.0262 - categorical_accuracy: 0.9928


[0.026231231167912483, 0.9927833080291748]